# Experiments with Mondial-Foreign Key using SQLAgent passing instances as examples - in GPT 3.5

In [2]:

from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from urllib.parse import quote  

from custom_sql_toolkit.custom_sql_database_toolkit import CustomSQLDatabaseToolkit
from custom_sql_toolkit.prompt import (
    CUSTOM_SQL_PREFIX,
    CUSTOM_FORMAT_INSTRUCTIONS,    
)
from langchain.callbacks import get_openai_callback
from langchain.agents import create_sql_agent

from langchain.agents.agent_types import AgentType

from dotenv import load_dotenv
import os
import sys
import json
import time

load_dotenv()

path = os.path.abspath('')
module_path = os.path.join(path,'..\..')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")

from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils



<>:24: SyntaxWarning: invalid escape sequence '\.'
<>:24: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Tahsin\AppData\Local\Temp\ipykernel_19324\4113694040.py:24: SyntaxWarning: invalid escape sequence '\.'
  module_path = os.path.join(path,'..\..')


c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..


C:\Users\Tahsin\AppData\Local\Temp\ipykernel_19324\4113694040.py:5: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from custom_sql_toolkit.custom_sql_database_toolkit import CustomSQLDatabaseToolkit


# Schema

In [3]:
SCHEMA = 'mondial_gpt'
PREFIX = 'mondial'
FILE_NAME_RESULT = f"results/8_sql_agent_chatgpt_{SCHEMA}_fk_instances.json"
NUMBER_OF_INSTANCES = 2


Functions for saving and reading generated queries

In [4]:
def save_queries(queries):
    data = {"queries":queries}
    with open(FILE_NAME_RESULT, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries():
    with open(FILE_NAME_RESULT, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

## Connection

In [5]:
json_file_path = f"../../datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)

db_connection

{'DB_HOST': 'localhost',
 'DB_PORT': '1522',
 'DB_USER_NAME': 'MONDIAL_GPT',
 'DB_PASS': 'TextDB123',
 'DB_NAME': '',
 'SQL_DRIVER': 'oracle+oracledb',
 'SERVICE_NAME': 'xepdb1',
 'SCHEMA': 'MONDIAL_GPT',
 'KEYWORD_SEARCH_API_URL': ''}

### Using SQLDatabase to get all the information from the database

In [6]:
db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables, samples=NUMBER_OF_INSTANCES)
db.get_table_names()

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'coordinates'
  self._metadata.reflect(
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'source'
  self._metadata.reflect(
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'estuary'
  self._metadata.reflect(


['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'economy',
 'encompasses',
 'ethnicgroup',
 'geo_desert',
 'geo_estuary',
 'geo_island',
 'geo_lake',
 'geo_mountain',
 'geo_river',
 'geo_sea',
 'geo_source',
 'island',
 'islandin',
 'ismember',
 'lake',
 'lakeonisland',
 'language',
 'located',
 'locatedon',
 'mergeswith',
 'mountain',
 'mountainonisland',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion',
 'river',
 'riveronisland',
 'riverthrough',
 'sea']

In [7]:
len(db.get_table_names())

46

In [8]:
cusmom_agent_prefix_prompt = CUSTOM_SQL_PREFIX
custom_format_instructions = CUSTOM_FORMAT_INSTRUCTIONS

In [9]:
print(custom_format_instructions)

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the SQL QUERY generated 
Final Answer: the SQL QUERY generated 


## Creating Chain to generate SQL

In [10]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", verbose=True)
toolkit = CustomSQLDatabaseToolkit(db=db.db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    prefix=cusmom_agent_prefix_prompt,
    format_instructions=custom_format_instructions,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)


C:\Users\Tahsin\AppData\Local\Temp\ipykernel_19324\1390492666.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", verbose=True)


In [11]:
with get_openai_callback() as cb:
    sql_query = agent_executor({"input":"What are the provinces with an area greater than 10000?"})
    print(cb.total_tokens)
    print(cb.prompt_tokens)
    print(cb.completion_tokens)
    print(cb.total_cost)
print(sql_query['output'])

C:\Users\Tahsin\AppData\Local\Temp\ipykernel_19324\2284450819.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  sql_query = agent_executor({"input":"What are the provinces with an area greater than 10000?"})




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe table 'province' seems relevant for this question. I should query the schema of the 'province' table.
Action: sql_db_schema
Action Input: province
CREATE TABLE province (
	name VARCHAR(50 CHAR) NOT NULL, 
	country VARCHAR(4 CHAR) NOT NULL, 
	population NUMBER, 
	area NUMBER, 
	capital VARCHAR(50 CHAR), 
	capprov VARCHAR(50 CHAR), 
	CONSTRAINT prkey PRIMARY KEY (name, country), 

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I can now construct the SQL query to find provinces with an area greater than 10000.
Final Answer: 
SELECT name, area
FROM province
WHERE area > 10000;

> Finished chain.
0
0
0
0.0
SELECT name, area
FROM province
WHERE area > 10000;


Total tokens: **3140**<br>
Prompt tokens: **2995**<br>
completion tokens: **145**<br>
Total cost: **0.0047825**<br>

## Preparing natural language queries to run in LLM

In [12]:

json_file_path = f"../../datasets/{PREFIX}/queries_{PREFIX}.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    queries = json.load(json_data, strict=False)
queries = queries['queries']
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': '',
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': '',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': '',
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': '',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': '',
  'type': 'complex'},
 {'id': '6',
  'question': 'What is the percentage of religious people are hindu in thailand?',
  'query_string': '',
  'type': 'complex'},
 {'id': '7',
  'question': 'List the number of provinces each river flows through.',
  'query_string': '',
  'type': 'medium'},
 {'id': '8',
  'question': 'Find all countries that became independent between 8/1/1910 and 8/1/1950.',
  'query_string': '',
  'type': 'complex'},
 {'id': '9',
  'que

# Running queries in LLM to generate SQL

In [13]:

# A cada X consultas, vai ser gerado um delay de 10s para evitar o bloqueio da API.
number_of_queries_to_delay = 2
count = 0
errors = []
pos = 0
for instance in queries:
    if count == number_of_queries_to_delay:
        count = 0
        time.sleep(10)
    try:
        with get_openai_callback() as cb:
            start_time = time.time()
            response = agent_executor({"input":instance["question"]})  
            end_time = time.time()
            sql_query = response['output']
            if "Agent stopped" in sql_query:
                errors.append(pos)
            else:
                instance["query_string"] = sql_query
                instance['total_tokens'] = cb.total_tokens
                instance['prompt_tokens'] = cb.prompt_tokens
                instance['completion_tokens'] = cb.completion_tokens
                instance['total_cost'] = cb.total_cost  
                instance['time'] = end_time - start_time
                print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
                save_queries(queries)
    except:
        errors.append(pos)
    count += 1
    pos+=1
queries



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe relevant tables for the area of Thailand are country and countrylocalname.
Action: sql_db_schema
Action Input: 'country, countrylocalname'Error: table_names {"countrylocalname'", "'country"} not found in databaseI made a typo in the input for sql_db_schema. Let me correct that and try again.
Action: sql_db_schema
Action Input: 'country, countrylocalname'Error: table_names {"cou

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


The relevant columns for the area of Thailand are 'name' and 'area' from the 'country' table.
Final Answer: SELECT name, area FROM country WHERE name = 'Thailand'

> Finished chain.
1 What is the area of Thailand? SELECT name, area FROM country WHERE name = 'Thailand' 6.967226505279541 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe 'province' table seems like the most relevant one to query for this question.
Action: sql

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I can now construct the SQL query to find provinces with an area greater than 10000.
Final Answer: 
SELECT name, area
FROM province
WHERE area > 10000;

> Finished chain.
2 What are the provinces with an area greater than 10000? SELECT name, area
FROM province
WHERE area > 10000; 2.5548079013824463 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe table 'language' seems relevant to the question.
Action: sql_db_schema
Actio

...

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the relevant tables and their schema. I can construct the SQL query to find the type of government in Iran.
Final Answer: 
SELECT government
FROM politics
WHERE country = 'IRAN';

> Finished chain.
99 What type of government is Iran? SELECT government
FROM politics
WHERE country = 'IRAN'; 2.9351184368133545 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe tables that seem relevant for this question are economy 

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 
Final Answer: SELECT industry FROM economy ORDER BY industry DESC

> Finished chain.
100 What is the percentage of industries in relation to Japan's economy? SELECT industry FROM economy ORDER BY industry DESC 3.424225330352783 0.0


[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT name, area FROM country WHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 0,
  'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_cost': 0.0,
  'time': 6.967226505279541},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name, area\nFROM province\nWHERE area > 10000;',
  'type': 'simple',
  'total_tokens': 0,
  'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_cost': 0.0,
  'time': 2.5548079013824463},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name\nFROM language\nWHERE country = 'PL'",
  'type': 'medium',
  'total_tokens': 0,
  'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_cost': 0.0,
  'time': 2.006389856338501},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth FROM lake WHERE name = 'Lake Kariba'",
  'type': 'simp

In [14]:
errors

[4, 7, 10, 19, 21, 28, 36, 49, 54, 56, 59, 61, 65, 67, 84, 87, 88, 93, 96]

Fixing Errors

In [16]:

# A cada X consultas, vai ser gerado um delay de 10s para evitar o bloqueio da API.
number_of_queries_to_delay = 15
count = 0

for pos in errors:
    instance = queries[pos]
    if count == number_of_queries_to_delay:
        count = 0
        time.sleep(10)

    with get_openai_callback() as cb:
        start_time = time.time()
        response = agent_executor({"input":instance["question"]})  
        end_time = time.time()
        sql_query = response['output']
        instance["query_string"] = sql_query
        instance['total_tokens'] = cb.total_tokens
        instance['prompt_tokens'] = cb.prompt_tokens
        instance['completion_tokens'] = cb.completion_tokens
        instance['total_cost'] = cb.total_cost  
        instance['time'] = end_time - start_time
        print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
    save_queries(queries)
    queries[pos] = instance
    count += 1
queries



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe relevant tables for this query are country and province.
Action: sql_db_schema
Action Input: 'country, province'Error: table_names {"province'", "'country"} not found in databaseI made a typo in the table names, I should correct it and try again.
Action: sql_db_schema
Action Input: 'country, province'Error: table_names {"province'", "'country"} not found in databaseI need to re

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: I can use the province table to count the total number of provinces in the Netherlands.
Action: custom_sql_db_query
Action Input: SELECT COUNT(name) FROM province WHERE country = 'NL'
Final Answer: SELECT COUNT(name) FROM province WHERE country = 'NL'
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

#### Fixing queries

In [ ]:

to_fix = [4, 7, 10, 19, 21, 28, 36, 49, 54, 56, 59, 61, 65, 67, 84, 87, 88, 93, 96]
for pos in to_fix:
    instance = queries[pos]
    q = read_queries()
    try:
        with get_openai_callback() as cb:
            start_time = time.time()
            response = agent_executor({"input":instance["question"]})  
            end_time = time.time()
            sql_query = response['output']
            if "Agent stopped" in sql_query:
                print("Agent stopped" + str(pos))
            else:
                instance["query_string"] = sql_query
                instance['total_tokens'] = cb.total_tokens
                instance['prompt_tokens'] = cb.prompt_tokens
                instance['completion_tokens'] = cb.completion_tokens
                instance['total_cost'] = cb.total_cost  
                instance['time'] = end_time - start_time
                print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
                q[pos] = instance
                save_queries(q)
    except:
        print("Agent error" + str(pos))



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe relevant tables for this query are country and province.
Action: sql_db_schema
Action Input: 'country, province'Error: table_names {"province'", "'country"} not found in databaseI made a typo in the table names. Let me correct that and try again.
Action: sql_db_schema
Action Input: 'country, province'Error: table_names {"province'", "'country"} not found in databaseI need to co

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


The relevant columns for the query are country.name and politics.independence.
Action: custom_sql_db_query
Action Input: SELECT country.name, politics.independence FROM country JOIN politics ON country.code = politics.country WHERE politics.independence BETWEEN '1910-08-01' AND '1950-08-01' ORDER BY politics.independenceError: (oracledb.exceptions.DatabaseError) ORA-01861: literal does not match format string
Help: https://docs.oracle.com/error-help/db/ora-01861/
[SQL: SELECT country.name, politics.independence FROM country JOIN politics ON country.code = politics.country WHERE politics.independence BETWEEN '1910-08-01' AND '1950-08-01' ORDER BY politics.independence]
(Background on this error at: https://sqlalche.me/e/20/4xp6)I need to adjust the date format in the query to match the database format.
Action: custom_sql_db_query
Action Input: SELECT country.name, politics.independence FROM country JOIN politics ON country.code = politics.country WHERE politics.independence BETWEEN TO_D

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 
Final Answer: SELECT COUNT(DISTINCT province.name) 
FROM province 
WHERE province.area > 1000 
AND province.country = 'Niger'

> Finished chain.
11 How many provinces have areas greater than 1000 in Niger? SELECT COUNT(DISTINCT province.name) 
FROM province 
WHERE province.area > 1000 
AND province.country = 'Niger' 5.0005762577056885 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaI shoul

...

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


The relevant columns for this query are the islands from the geo_island table and the elevation from the lake table. I should now construct the SQL query.
Final Answer: 
SELECT gisland.island
FROM geo_island gisland
JOIN geo_lake glake ON gisland.province = glake.province AND gisland.country = glake.country
WHERE glake.elevation > 100;

> Finished chain.
94 List the islands that belong to a lake with an elevation greater than 100 SELECT gisland.island
FROM geo_island gisland
JOIN geo_lake glake ON gisland.province = glake.province AND gisland.country = glake.country
WHERE glake.elevation > 100; 4.1425676345825195 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, is

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I need to join the country and citypops tables on the country code to get the name of the countries, the average elevation, and sum of the population of their cities.
Action: custom_sql_db_query
Action Input: SELECT c.name AS country_name, AVG(cp.population) AS avg_city_population, SUM(cp.population) AS total_city_population
FROM country c
JOIN citypops cp ON c.code = cp.country
GROUP BY c.name
ORDER BY total_city_population DESC;Error: (oracledb.exceptions.DatabaseError) ORA-00933: SQL command not properly ended
Help: https://docs.oracle.com/error-help/db/ora-00933/
[SQL: SELECT c.name AS country_name, AVG(cp.population) AS avg_city_population, SUM(cp.population) AS total_city_population
FROM country c
JOIN citypops cp ON c.code = cp.country
GROUP BY c.name
ORDER BY total_city_population DESC;]
(Background on this error at: https://sqlalche.me/e/20/4xp6)I need to fix the query syntax before running it.
Action: custom_sql_db_query
Action Input: SELECT c.name AS country_name, AVG(cp.pop

: 